## ENS Owners

In [1]:
import pandas as pd
from numpy import ndarray
from ens import ENS
from web3 import Web3, HTTPProvider
from tqdm.notebook import tqdm
import ipfshttpclient
import content_hash
import json
from pprint import pprint

from config import ETH_URL, MN
from src.contract_utils import execute_contract_bash


EXTRACT_NAMES = False
EXTRACT_ENS_DATA = False
AGGREGATE_ENS_DATA = True

tqdm.pandas()

w3 = Web3(HTTPProvider(ETH_URL))
ns = ENS.fromWeb3(w3)
ipfs_client = ipfshttpclient.connect()

/Users/sergenedashkovsky/Library/Python/3.9/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.15.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### ENS Names

In [2]:
ens_names = pd.read_csv('data/ens_names.csv')
ens_names_list = ens_names.name.to_list()
ens_names_list[:10]

['ethercryptocurrency',
 'ufcfighters',
 'coinapp',
 '2rickychan',
 'lootvault',
 'arthurs',
 'feelsgoodman',
 'doodzuki',
 'fattigerworkshop',
 'bioart']

### Items with content CIDs

In [3]:
ens_data_with_cid_df = pd.read_csv('data/ens_data_with_content_cid.csv', index_col=0)
ens_data_with_cid_df.head(10)

,name,owner,resolver_address,associated_address,contenthash,content_cid,name_cid
66331,0-1337.eth,0x8F82De72659D5a1670d3Db7C440A8E0a6E925E02,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,NaN,e30101701220d6f0e0d157170f52052e3f81bdafafb2e3...,QmcogzR9ouQJDgsiaTWWze4ndMLkn6gewRqKVWeKXVozh7,QmUwjXxYcgk1j2Eh7jiVtgSGn3hmLHWoqNDLTonts6hJV8
71898,0000x.eth,0x77C9e49073d9830723195f3788570c2c36c97688,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x77C9e49073d9830723195f3788570c2c36c97688,e301017012203c7939754cbfabd4a7932a740b2af3ced2...,QmSQiPAPaT5E9y41VgHhc5kFRhRFSvZ95d5sBzH6VJwdPo,QmVzMe3mUGLB68Lz5wpQcfjab8akjQj7SdoE1ioAzfiw6E
75750,0008a.eth,0x88441DCD4Bc985965e69ecf5A454B04DC84eD876,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x88441DCD4Bc985965e69ecf5A454B04DC84eD876,e30101701220cdaf3eb20495dacca5225a9a0838207af7...,QmcBZGuZCQsd5zuumVRuLhXnhkWjWJiXtmcE9XQVsAXV8N,Qmawaesea9bHkCMmZupyMn5pbVgAJjKVM3MS2rUqZ1GhKf
78535,001x0.eth,0x338571a641d8c43F9e5A306300c5d89E0CB2CFAF,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x338571a641d8c43F9e5A306300c5d89E0CB2CFAF,e301017012209668df055d7566e1631a6db9d66015204a...,QmYTncViYfeLbyHS6EECfUoRW3kjqrbiUmKbZLZEXUmA6R,QmXk8AcZzo88TBa1TiJT3uLzcKnTY8DTyVa7euFmMh9qYw
79530,002511.eth,0x4E99f6Fdb8B5a3A292aa2194c970c2A612469Fe2,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x4E99f6Fdb8B5a3A292aa2194c970c2A612469Fe2,e301017012205952f152144216dfc3267c1f44130f1235...,QmUMLME8Lkipg1cqw8m3pwpWHhRN34ioFEbjbhw46oqkN2,QmNbqwb5T2oWTUjWMRWxUBAGYUvy3v6Ta7SHaz83Yo9TT6
79618,00258.eth,0x9e6D2A36fa88E5123fD09d22F00e4E9ca8669999,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x9e6D2A36fa88E5123fD09d22F00e4E9ca8669999,e301017012200429cb961fb23fd71947c477c881f63671...,QmNcuJbp2YLK6BmhdTniVsjjfxcgP16dGBKZeUHQQ6vHqB,QmUhnpZg3Wvz3eSTrAj66xVa3xrKb62CF63V5zWwu6Z3pa
89735,0089.eth,0xFebfecE019b5E2e42C3839dCbEA87EEDac9ac365,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0xFebfecE019b5E2e42C3839dCbEA87EEDac9ac365,e301017012200676503f43d887c1e1d20c23d988d4d941...,QmNmsnw5FpncDeUvJ3ZRFY3YtvEYWoCHWvgjEwjV41gUjM,QmSTGTAnKY5n3Xz4DEY1oWEmrVPuAXTp469L4bGVWyiX12
96266,0123a.eth,0x9aE6B470D238a4ab42cAc5fEB27CAA5A19F89510,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x9aE6B470D238a4ab42cAc5fEB27CAA5A19F89510,e30101701220c801aa770d8313b86fe43cb08472166a71...,QmboPirQwrzimcSfGbeRHxySy4sH19ZmoJywVSf7bH7n8T,QmaUedgdJdVrzAEGud28YFNAv6cuwnbdxBgG4WktfZNpSZ
97207,014°.eth,0x70635102CF3B7D32FE0a1221b412Cb4b172A2b2f,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x70635102CF3B7D32FE0a1221b412Cb4b172A2b2f,e3010170122086f1db4d21efaa7220ad3ec4460ac2b97b...,QmXRREtZRWsppmHwJqFYW9RfazaWh5mob5iRBgX1GLtcF9,QmWRhM2YPtfZbLNaW4Fk2JLuQBBFWNXDzWyzrYazYieE4r
99044,01x01.eth,0x8F82De72659D5a1670d3Db7C440A8E0a6E925E02,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x8F82De72659D5a1670d3Db7C440A8E0a6E925E02,e30101701220d6f0e0d157170f52052e3f81bdafafb2e3...,QmcogzR9ouQJDgsiaTWWze4ndMLkn6gewRqKVWeKXVozh7,QmSF6D6bPeR4FF8MiJgYYY5prufPh3B17g2bmpxZkFzWqy


#### ENS Namespace Contracts with CIDs only
- name -> owner
- owner -> name
- name -> associated_address
- associated_address -> name
- name -> cid
- cid -> name

In [4]:
name_owner_cnt = len(ens_data_with_cid_df[~ens_data_with_cid_df.owner.isna()][['name', 'owner']])
name_associated_address_cnt = len(ens_data_with_cid_df[~ens_data_with_cid_df.associated_address.isna()])
name_contenthash_cnt = len(
    ens_data_with_cid_df)
name_cid_cnt = len(
    ens_data_with_cid_df)
print(f"name -> owner:              {name_owner_cnt:>7,} ")
print(f"owner -> name:              {name_owner_cnt:>7,} \n")
print(f"name -> associated_address: {name_associated_address_cnt:>7,} ")
print(f"associated_address -> name: {name_associated_address_cnt:>7,} \n")
print(f"name -> cid:                {name_cid_cnt:>7,} ")
print(f"cid -> name:                {name_cid_cnt:>7,} \n")
print(f"Total cyberlinks:           {(name_owner_cnt + name_associated_address_cnt + name_cid_cnt) * 2 :>7,}")

name -> owner:                5,607 
owner -> name:                5,607 

name -> associated_address:   5,269 
associated_address -> name:   5,269 

name -> cid:                  5,607 
cid -> name:                  5,607 

Total cyberlinks:            32,966


### Upload to Subgraphs
Example of bash execution:
```bash
export EXECUTE = '{"cyberlink": {"links":[{"from":"Qmex9kMduo4U8k8AB4otfqgHRehUNs8YCksUjBQXW9WeQi","to":"QmSkCKtKSmcsa2r5ju5SnHiLysktDqFWzTHdG4QS6yTeCH"}]}}'
pussy tx wasm execute pussy1hrpna9v7vs3stzyd4z3xf00676kf78zpe2u5ksvljswn2vnjp3ysehgnqr $EXECUTE --from testload --chain-id space-pussy --gas 3000000 --broadcast-mode block  $SNODE
```

In [5]:
def create_cyberlinks(
        cyberlinks: ndarray,
        contract_address: str = 'pussy1hrpna9v7vs3stzyd4z3xf00676kf78zpe2u5ksvljswn2vnjp3ysehgnqr',
        batch_size: int = 1,
        from_address: str = 'testload',
        gas: int = 10_000_000) -> tuple[list, bool]:

    def _batch(x, batch_size: int) -> list:
        return [x[_i:_i + batch_size] for _i in range(0, len(x), batch_size)]

    _res = []
    _res_status = True
    for _ens_data_with_cid_item in tqdm(_batch(cyberlinks, batch_size)):
        _links_item = ', '.join(
            f'{{"from": "{_item[0]}", "to": "{_item[1]}"}}' for _item in _ens_data_with_cid_item)
        # print(f'''{{"cyberlink": {{"links": [{_links_item}]}}}}''')
        _res_item = execute_contract_bash(
            execute_query=f'''{{"cyberlink": {{"links": [{_links_item}]}}}}''',
            contract_address=contract_address,
            from_address=from_address,
            display_data=False,
            gas=gas
        )
        _res.append(_res_item)
        if json.loads(json.loads(_res_item)['raw_log'])[0]['events'][0]['type'] == 'execute':
            print(f'Error in {len(_res) - 1}\n{{"cyberlinks": [{_links_item}]}}\n')
            _res_status = False
            break
    return _res, _res_status

#### Upload to `name -> content CID` Subgraph contract

In [ ]:
res, res_status = create_cyberlinks(cyberlinks=ens_data_with_cid_df[['name_cid', 'content_cid']].values[:])
if not res_status:
    pprint(json.loads(json.loads(res[-1])['raw_log'])[0])